In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import os


from sklearn import preprocessing
#from sklearn import StandardScaler
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from scipy import stats



from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
# from yellowbrick.classifier import DiscriminationThreshold



/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv('train_jqd04QH.csv')



In [3]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [4]:
df.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
dtype: int64

In [5]:
df.shape

(18359, 14)

In [6]:
df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                      int64
dtype: object

## Mutating NA's

In [7]:
# Changing Categorical Values to Numerical

experience = {'Has relevent experience':1,'No relevent experience':0}
df['relevent_experience'] = df['relevent_experience'].map(experience)

#Gender
df['gender'].fillna('Male',inplace= True)

#Enrolled University
df['enrolled_university'].fillna('no_enrollment', inplace= True)

#Education Level
df['education_level'].fillna('Graduate', inplace = True)
education = {'Phd':5,'Masters':4,'Graduate':3,'High School':2,'Primary School':1}
df['education_level'] = df['education_level'].map(education)

#Major Discipline
df['major_discipline'].fillna('STEM', inplace = True)

# Experience
df['experience'] = df['experience'].astype('str')
df['experience'].replace({'>20':'25','<1':'0.5'},inplace= True)
df['experience'] = df['experience'].astype('float')
df['experience'].fillna((df['experience'].median()),inplace= True)

# Last New Job
df['last_new_job'] = df['last_new_job'].astype('str')
df['last_new_job'].replace({'never':'0','>4':'5'},inplace= True)
df['last_new_job'] = df['last_new_job'].astype('float')
df['last_new_job'].fillna(1.0,inplace= True)

# Company Type
# df['company_type'] = df['company_type'].astype('str')
df['company_type'].fillna('Unknown',inplace= True)

#Company Size
df['company_size'] = df['company_size'].astype('str')
df['company_size'].replace({'10/49':'10-49'},inplace= True)
df['company_size'].fillna('Unknown',inplace = True)




In [8]:
df.isnull().sum()

enrollee_id               0
city                      0
city_development_index    0
gender                    0
relevent_experience       0
enrolled_university       0
education_level           0
major_discipline          0
experience                0
company_size              0
company_type              0
last_new_job              0
training_hours            0
target                    0
dtype: int64

In [9]:
df.dtypes

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience         int64
enrolled_university        object
education_level             int64
major_discipline           object
experience                float64
company_size               object
company_type               object
last_new_job              float64
training_hours              int64
target                      int64
dtype: object

In [10]:
df['gender'].value_counts()

Male      16982
Female     1188
Other       189
Name: gender, dtype: int64

In [11]:
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,1,no_enrollment,3,STEM,3.0,100-500,Pvt Ltd,1.0,106,0
1,29166,city_83,0.923,Male,1,no_enrollment,3,STEM,14.0,<10,Funded Startup,1.0,69,0
2,46,city_16,0.910,Male,1,no_enrollment,3,STEM,6.0,50-99,Public Sector,2.0,4,0
3,18527,city_64,0.666,Male,1,no_enrollment,3,STEM,14.0,50-99,Pvt Ltd,1.0,26,0
4,21751,city_100,0.887,Male,0,no_enrollment,4,STEM,8.0,nan,Unknown,2.0,88,1


In [12]:
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
df['city'] = labelencoder.fit_transform(df['city'])
df.head()


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,41,0.689,Male,1,no_enrollment,3,STEM,3.0,100-500,Pvt Ltd,1.0,106,0
1,29166,112,0.923,Male,1,no_enrollment,3,STEM,14.0,<10,Funded Startup,1.0,69,0
2,46,48,0.910,Male,1,no_enrollment,3,STEM,6.0,50-99,Public Sector,2.0,4,0
3,18527,93,0.666,Male,1,no_enrollment,3,STEM,14.0,50-99,Pvt Ltd,1.0,26,0
4,21751,2,0.887,Male,0,no_enrollment,4,STEM,8.0,nan,Unknown,2.0,88,1


In [13]:
train = pd.get_dummies(df,columns= ['gender','enrolled_university','major_discipline','company_size','company_type'])

In [14]:
train.shape

(18359, 37)

In [15]:
train.head()

,enrollee_id,city,city_development_index,relevent_experience,education_level,experience,last_new_job,training_hours,target,gender_Female,...,company_size_5000-9999,company_size_<10,company_size_nan,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
0,23798,41,0.689,1,3,3.0,1.0,106,0,0,...,0,0,0,0,0,0,0,0,1,0
1,29166,112,0.923,1,3,14.0,1.0,69,0,0,...,0,1,0,0,1,0,0,0,0,0
2,46,48,0.910,1,3,6.0,2.0,4,0,0,...,0,0,0,0,0,0,0,1,0,0
3,18527,93,0.666,1,3,14.0,1.0,26,0,0,...,0,0,0,0,0,0,0,0,1,0
4,21751,2,0.887,0,4,8.0,2.0,88,1,0,...,0,0,1,0,0,0,0,0,0,1


In [16]:
train.set_index(train['enrollee_id'], inplace= True)
train.drop('enrollee_id', axis = 1, inplace = True)
train.head()

,city,city_development_index,relevent_experience,education_level,experience,last_new_job,training_hours,target,gender_Female,gender_Male,...,company_size_5000-9999,company_size_<10,company_size_nan,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
enrollee_id,,,,,,,,,,,,,,,,,,,,,
23798,41,0.689,1,3,3.0,1.0,106,0,0,1,...,0,0,0,0,0,0,0,0,1,0
29166,112,0.923,1,3,14.0,1.0,69,0,0,1,...,0,1,0,0,1,0,0,0,0,0
46,48,0.910,1,3,6.0,2.0,4,0,0,1,...,0,0,0,0,0,0,0,1,0,0
18527,93,0.666,1,3,14.0,1.0,26,0,0,1,...,0,0,0,0,0,0,0,0,1,0
21751,2,0.887,0,4,8.0,2.0,88,1,0,1,...,0,0,1,0,0,0,0,0,0,1


In [17]:
train.rename({'company_size_<10':'company_size_below_10'},axis = 1, inplace = True)

## Adding some more features

In [18]:
train['Small_city'] = np.where((train['city_development_index'] > 0.448) & (train['city_development_index'] < 0.7960), 1,0)
train['Medium_city'] = np.where((train['city_development_index'] > 0.7960) & (train['city_development_index'] < 0.91), 1,0)
train['Big_city'] = np.where((train['city_development_index'] > 0.91), 1,0)

In [20]:
train.to_csv('train_preprocessed.csv')

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections

In [22]:
X,y = train.drop(['target','city'],axis=1),train['target']


In [23]:
X['city_development_index'].describe()

count    18359.000000
mean         0.847140
std          0.110189
min          0.448000
25%          0.796000
50%          0.910000
75%          0.920000
max          0.949000
Name: city_development_index, dtype: float64

In [24]:
X[['Small_city','Medium_city','Big_city','city_development_index']].head()

,Small_city,Medium_city,Big_city,city_development_index
enrollee_id,,,,
23798,1,0,0,0.689
29166,0,0,1,0.923
46,0,0,0,0.910
18527,1,0,0,0.666
21751,0,1,0,0.887


In [25]:
X.shape

(18359, 37)

## Random Foresttt

In [46]:
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
model = rf.fit(X, y);

In [26]:
test = pd.read_csv('test_KaymcHn.csv')

In [27]:
test.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,16548,city_33,0.448,NaN,No relevent experience,Full time course,Graduate,STEM,<1,1000-4999,Public Sector,NaN,15
1,12036,city_28,0.939,Male,No relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,94
2,11061,city_103,0.920,Male,No relevent experience,Full time course,Graduate,STEM,3,NaN,NaN,1,17
3,5032,city_104,0.924,Male,No relevent experience,no_enrollment,Phd,STEM,>20,50-99,Pvt Ltd,2,76
4,17599,city_77,0.830,Male,Has relevent experience,no_enrollment,Graduate,STEM,6,<10,Pvt Ltd,2,65


In [28]:
test.shape

(15021, 13)

In [29]:
test.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    3388
relevent_experience          0
enrolled_university        279
education_level            395
major_discipline          2393
experience                  44
company_size              4051
company_type              4330
last_new_job               304
training_hours               0
dtype: int64

In [30]:
test.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours'],
      dtype='object')

In [31]:
def preprocess_data(df):
    
    experience = {'Has relevent experience':1,'No relevent experience':0}
    df['relevent_experience'] = df['relevent_experience'].map(experience)

    #Gender
    df['gender'].fillna('Male',inplace= True)

    #Enrolled University
    df['enrolled_university'].fillna('no_enrollment', inplace= True)

    #Education Level
    df['education_level'].fillna('Graduate', inplace = True)
    education = {'Phd':5,'Masters':4,'Graduate':3,'High School':2,'Primary School':1}
    df['education_level'] = df['education_level'].map(education)

    #Major Discipline
    df['major_discipline'].fillna('STEM', inplace = True)

    # Experience
    df['experience'] = df['experience'].astype('str')
    df['experience'].replace({'>20':'25','<1':'0.5'},inplace= True)
    df['experience'] = df['experience'].astype('float')
    df['experience'].fillna((df['experience'].median()),inplace= True)

    # Last New Job
    df['last_new_job'] = df['last_new_job'].astype('str')
    df['last_new_job'].replace({'never':'0','>4':'5'},inplace= True)
    df['last_new_job'] = df['last_new_job'].astype('float')
    df['last_new_job'].fillna(1.0,inplace= True)

    # Company Type
    # df['company_type'] = df['company_type'].astype('str')
    df['company_type'].fillna('Unknown',inplace= True)

    #Company Size
    df['company_size'] = df['company_size'].astype('str')
    df['company_size'].replace({'10/49':'10-49'},inplace= True)
    df['company_size'].fillna('Unknown',inplace = True)
    
#     labelencoder = LabelEncoder()
#     # Assigning numerical values and storing in another column
#     df['city'] = labelencoder.fit_transform(df['city'])

    # Dropping City column for this model
    
    df.drop('city',axis = 1, inplace = True)
    
    df['Small_city'] = np.where((df['city_development_index'] > 0.448) & (df['city_development_index'] < 0.7960), 1,0)
    df['Medium_city'] = np.where((df['city_development_index'] > 0.7960) & (df['city_development_index'] < 0.91), 1,0)
    df['Big_city'] = np.where((df['city_development_index'] > 0.91), 1,0)
    
    test = pd.get_dummies(df,columns= ['gender','enrolled_university','major_discipline','company_size','company_type'])
    
    
    test.set_index(test['enrollee_id'], inplace= True)
    test.drop('enrollee_id', axis = 1, inplace = True)
    
    test.rename({'company_size_<10':'company_size_below_10'},axis = 1, inplace = True)
    
    return test



In [32]:
test_data = preprocess_data(df= test)

In [33]:
test_data.head()

,city_development_index,relevent_experience,education_level,experience,last_new_job,training_hours,Small_city,Medium_city,Big_city,gender_Female,...,company_size_5000-9999,company_size_below_10,company_size_nan,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
enrollee_id,,,,,,,,,,,,,,,,,,,,,
16548,0.448,0,3,0.5,1.0,15,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
12036,0.939,0,3,25.0,1.0,94,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
11061,0.920,0,3,3.0,1.0,17,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
5032,0.924,0,5,25.0,2.0,76,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
17599,0.830,1,3,6.0,2.0,65,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0


In [34]:
X.shape, test_data.shape, test.shape

((18359, 37), (15021, 37), (15021, 15))

In [35]:
test_data.to_csv('test_preprocessed.csv')

In [55]:
pred = model.predict(test_data)

In [58]:
test_data['target'] = pred

In [60]:
test_data.reset_index(inplace=True)

In [62]:
submission = test_data[['enrollee_id','target']]
submission.to_csv('XGB_baseline.csv')

## Logistic Baseline

In [84]:
lr = LogisticRegression()

In [85]:
lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
from sklearn.model_selection import GridSearchCV


# Logistic Regression 
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X, y)
# We automatically get the logistic regression with the best parameters.
log_reg = grid_log_reg.best_estimator_
print(log_reg)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [87]:
log_reg

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [88]:
lr_model = log_reg.fit(X,y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [116]:
pred = lr_model.predict_proba(test_data)[:,1]

In [117]:
pred

array([0.19979583, 0.08356999, 0.15522254, ..., 0.13044306, 0.15016992,
       0.15736269])

In [106]:
test_data['target'] = pred
test_data.reset_index(inplace=True)

submission = test_data[['enrollee_id','target']]
submission.to_csv('Logistic baseline 2.csv')

In [120]:
test_data.to_csv('test_preprocessed.csv')
X.to_csv('train_preprocessed.csv')

In [119]:
X.shape

(18359, 37)

In [122]:
train.shape, test_data.shape

((18359, 36), (15021, 37))